In [69]:
!pip install xgboost
import pandas as pd
import xgboost

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [70]:


# Load the data from the 'data_processed.csv' file 
X_train = pd.read_csv('X_train_C.csv', low_memory=False)
X_test = pd.read_csv('X_test_C.csv', low_memory=False)
Y_train = pd.read_csv('Y_train_C.csv', low_memory=False)
Y_test = pd.read_csv('Y_test_C.csv', low_memory=False)



Y_test

,Nail spacing [cm],Number sheathing panels,Number end studs,Total number studs,HoldDown Model,Tx(s),Ty(s)
0,5,2,3,16,4.4450,0.445221,0.384104
1,10,1,4,14,19.0000,0.615371,0.421098
2,10,1,4,14,13.0000,0.706989,0.703002
3,5,2,3,8,9.0000,0.664038,0.667764
4,10,1,1,16,1.5875,0.698229,0.591958
...,...,...,...,...,...,...,...
12448,5,2,4,10,20.0000,0.421084,0.520357
12449,15,1,1,8,1.2700,0.964935,0.716120
12450,15,1,3,8,4.0000,0.646026,0.649899
12451,10,1,4,11,21.0000,0.642927,0.587151


The First 3 collumns of the dataset can be treated as catagorical data :

Nai spacing can only take 3 values ( 5 ,10 ,15) , even if there's an order relationship between those 3 values , we can still try to fit a Classification model treating them as categorical values

Number sheathing panels can only takes 2 values ( 1 or 2 )

Number Ends Studs can only take 6 values : (1 2 3 4 5 6)

In [71]:
categorical_columns = ["Nail spacing [cm]","Number sheathing panels","Number end studs"]
regression_columns = ["Total number studs","HoldDown Model","Tx(s)","Ty(s)"]



Y_train_cat = pd.get_dummies(Y_train[categorical_columns],columns=categorical_columns)
Y_train_reg = Y_train[regression_columns]

Y_test_cat = pd.get_dummies(Y_test[categorical_columns],columns=categorical_columns)
Y_test_reg = Y_test[regression_columns]

In [72]:
import numpy as np

#Seems like it doesn't improve the model accuracy 

def Nail_spacing_mapping(x): #- Nail spacing can be only {5 10 15} cm
    return np.where(x < 5, 5, np.where(x > 15, 15, x))


def sheathing_panels_mapping(x): #- Number of sheathing panels can be only {1 2} 
    return np.where(x < 1, 1, np.where(x > 2, 2, x))
def Y_pred_mapping(Y_pred):


    Y_pred[:,0]  = Nail_spacing_mapping(Y_pred[:,0])
    Y_pred[:,1]  = sheathing_panels_mapping(Y_pred[:,1])
    #Y_pred[:,2] = np.round(Y_pred[:,2]) #Number of end studs is an integer
    #Y_pred[:,3] = np.round(Y_pred[:,3])#Total number studs is also an integer 



In [73]:
#RUN in 20 Min 

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier



# Hyperparameters to try
alpha_values = np.arange(0.1, 11.0, 0.5)
n_estimators_values = np.arange(10, 30, 10)
n_neighbors = np.arange(1,10,1)

# Define a list of estimators with their respective hyperparameters
estimators = [

    


    (BaggingRegressor(), {'estimator__n_estimators': n_estimators_values}), #0.37110787901763215
    (GradientBoostingRegressor(),{'estimator__n_estimators': n_estimators_values}),# 1.6587493376697904 4 Min 44
    (KNeighborsRegressor(),{'estimator__n_neighbors':n_neighbors}), #1.8000806798211735

  
    (RandomForestRegressor(), {'estimator__n_estimators': n_estimators_values})
]#

# Different k values for k-fold cross-validation

cv_values = [5]

# Store the best model, its parameters, and performance
best_model = None
best_cv = None
best_mse = float('inf')
all_results = []
Use_Y_pred_mapping_tab = [True,False]
used_y_pred_mapping =False

# Iterate over different values of k for cross-validation

for cv in cv_values:
    results = []

    # Iterate through the estimators and their hyperparameters
    for estimator, param_grid in estimators:
        regressor = MultiOutputRegressor(estimator)
        grid_search = GridSearchCV(regressor, param_grid, cv=cv, scoring='neg_mean_squared_error')
        grid_search.fit(X_train, Y_train_reg)
        

        Y_pred = grid_search.predict(X_test)


        mse = mean_squared_error(Y_test_reg, Y_pred)
        
        results.append({
            'CV': cv,
            'Estimator': estimator.__class__.__name__,
            'Best Parameters': grid_search.best_params_,
            'Mean Squared Error': mse,

        })
        
        if mse < best_mse:
            best_mse = mse
            best_cv = cv
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_
            

    all_results.extend(results)

# Convert all results to a DataFrame
all_results_df = pd.DataFrame(all_results)

# Display the results
print(all_results_df)

# Display the overall best model and its performance
print("\nOverall Best Model Results:")
print(f"Best CV: {best_cv}")
print(f"Best Mean Squared Error: {best_mse}")
print(f"Best Hyperparameters: {best_params}")
print(f"Best Model: {best_model}")

   CV                  Estimator                  Best Parameters  \
0   5           BaggingRegressor  {'estimator__n_estimators': 20}   
1   5  GradientBoostingRegressor  {'estimator__n_estimators': 20}   
2   5        KNeighborsRegressor    {'estimator__n_neighbors': 2}   
3   5      RandomForestRegressor  {'estimator__n_estimators': 20}   

   Mean Squared Error  
0            0.086747  
1            3.258224  
2            1.114332  
3            0.088230  

Overall Best Model Results:
Best CV: 5
Best Mean Squared Error: 0.08674677491711291
Best Hyperparameters: {'estimator__n_estimators': 20}
Best Model: MultiOutputRegressor(estimator=BaggingRegressor(n_estimators=20))


In [74]:
all_results_df = pd.DataFrame(all_results)
all_results_df

,CV,Estimator,Best Parameters,Mean Squared Error
0,5,BaggingRegressor,{'estimator__n_estimators': 20},0.086747
1,5,GradientBoostingRegressor,{'estimator__n_estimators': 20},3.258224
2,5,KNeighborsRegressor,{'estimator__n_neighbors': 2},1.114332
3,5,RandomForestRegressor,{'estimator__n_estimators': 20},0.088230


In [75]:
""" RUN WITH ALL REGRESSIONS

   CV                  Estimator                  Best Parameters  \
0   5           BaggingRegressor  {'estimator__n_estimators': 75}   
1   5  GradientBoostingRegressor  {'estimator__n_estimators': 95}   
2   5      RandomForestRegressor  {'estimator__n_estimators': 80}   

   Mean Squared Error  
0            0.333335  
1            1.680206  
2            0.334892  

Overall Best Model Results:
Best CV: 5
Best Mean Squared Error: 0.33333456516122734
Best Hyperparameters: {'estimator__n_estimators': 75}
Best Model: MultiOutputRegressor(estimator=BaggingRegressor(n_estimators=75))





"""


" RUN WITH ALL REGRESSIONS\n\n   CV                  Estimator                  Best Parameters  0   5           BaggingRegressor  {'estimator__n_estimators': 75}   \n1   5  GradientBoostingRegressor  {'estimator__n_estimators': 95}   \n2   5      RandomForestRegressor  {'estimator__n_estimators': 80}   \n\n   Mean Squared Error  \n0            0.333335  \n1            1.680206  \n2            0.334892  \n\nOverall Best Model Results:\nBest CV: 5\nBest Mean Squared Error: 0.33333456516122734\nBest Hyperparameters: {'estimator__n_estimators': 75}\nBest Model: MultiOutputRegressor(estimator=BaggingRegressor(n_estimators=75))\n\n\n\n\n\n"

In [76]:
##Run in 10 m

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.multioutput import MultiOutputRegressor
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


# Hyperparameters to try
alpha_values = np.arange(0.1, 11.0, 0.5)
n_estimators_values = np.arange(10, 31, 10)
n_neighbors = np.arange(1,10,1)

# Define a list of estimators with their respective hyperparameters
estimators = [
    (LogisticRegression(max_iter=1000),{}),
    (RandomForestClassifier(),{'estimator__n_estimators': n_estimators_values}),#TODO add hyperparameters
    (DecisionTreeClassifier(),{}),
    (KNeighborsClassifier(),{'estimator__n_neighbors':n_neighbors})

]#

# Different k values for k-fold cross-validation

cv_values = [5]

# Store the best model, its parameters, and performance
best_model = None
best_cv = None
best_f1=0
best_accuracy=0
all_results = []

# Iterate over different values of k for cross-validation
for cv in cv_values:
    results = []

    # Iterate through the estimators and their hyperparameters
    for estimator, param_grid in estimators:
        regressor = MultiOutputClassifier(estimator)
        grid_search = GridSearchCV(regressor, param_grid, cv=cv)
        grid_search.fit(X_train, Y_train_cat)
        

        Y_pred = grid_search.predict(X_test)
  


        accuracy = accuracy_score(Y_test_cat, Y_pred)
        f1 = f1_score(Y_test_cat, Y_pred,average="micro")
        results.append({
            'CV': cv,
            'Estimator': estimator.__class__.__name__,
            'Best Parameters': grid_search.best_params_,
            'best_score ': grid_search.best_score_,
            'Best accuracy':accuracy,

        })
        
        if accuracy > best_accuracy:
            best_cv = cv
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_
            best_accuracy = accuracy

    all_results.extend(results)

# Convert all results to a DataFrame
all_results_df = pd.DataFrame(all_results)

# Display the results
print(all_results_df)

# Display the overall best model and its performance
print("\nOverall Best Model Results:")
print(f"Best CV: {best_cv}")
print(f"Best Hyperparameters: {best_params}")
print(f"Best Model: {best_model}")
print(f"Best accuracy: {best_accuracy}")


   CV               Estimator                  Best Parameters  best_score   \
0   5      LogisticRegression                               {}     0.412849   
1   5  RandomForestClassifier  {'estimator__n_estimators': 30}     0.763381   
2   5  DecisionTreeClassifier                               {}     0.831781   
3   5    KNeighborsClassifier    {'estimator__n_neighbors': 3}     0.680646   

   Best accuracy  
0       0.409781  
1       0.771220  
2       0.863567  
3       0.703766  

Overall Best Model Results:
Best CV: 5
Best Hyperparameters: {}
Best Model: MultiOutputClassifier(estimator=DecisionTreeClassifier())
Best accuracy: 0.8635670119649883
